<a href="https://colab.research.google.com/github/LucasVD23/Medical_QA_with_LLM/blob/main/medqa_with_multiagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import os
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Defining important funcions and classes

## RAG


In [2]:

def get_textbooks(dataset_path):
    textbooks = []
    for book in os.listdir(dataset_path + 'textbooks/en/'):
        file = open(dataset_path + 'textbooks/en/' + book, 'r')
        textbooks.append('\n'.join(file.readlines()))

    return textbooks


In [3]:
class RAG():
    def __init__(self, model_name, vector_store_path) -> None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model_kwargs = {'device': device}
        self.embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs)
        self.vector_store_path = vector_store_path


    def create_vectorstore(self,knowledge_base_path, text_splitter = None):

        if text_splitter is None:
          text_splitter = SemanticChunker(self.embedding_model)
        textbooks = get_textbooks(knowledge_base_path)
        chunks = text_splitter.create_documents(textbooks)

        faiss_store  = FAISS.from_documents(chunks, embedding=self.embedding_model)
        faiss_store.save_local(self.vector_store_path)

    def retrieve_documents(self, query, num_docs = 5):
        faiss_store = FAISS.load_local(self.vector_store_path,
                                self.embedding_model,
                                allow_dangerous_deserialization = True)

        similar_docs = faiss_store.similarity_search(query, k=num_docs)
        similar_docs = [doc.page_content for doc in similar_docs]

        return similar_docs

## OpenAI API and RAGAS Metrics


In [65]:
from abc import ABC, abstractmethod
import re
from openai import OpenAI


client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

class GPT():
  def __init__(self, model = 'gpt-4o-mini'):

    self.client = OpenAI()
    self.model = model

  def get_answer(self, prompt):
    response =  client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return response.choices[0].message.content.strip()
gpt4o = GPT()

#Classe pai que as métricas irão herdar
class Metric(ABC):

    def __init__(self, model):
        self.model = model

    #Obriga a implementação do evaluate
    @abstractmethod
    def evaluate(self, question, context, answer):
        raise NotImplementedError

class Faithulness(Metric):
    def __init__(self, model):  # Add model as an argument
        super().__init__(model)  # Pass the model to the parent class

    def __process_statements(self, statements):
        """
        Função para extrair afirmações da LLM
        """
        #Expressão regular para capturar statemnts
        pattern = r"Statement \d+:\s*(.+)"

        matches = re.findall(pattern, statements)

        #Caso a resposta não esteja de acordo com o formato especificado
        if not matches:
          print(f"No statements found in the response: {statements}")

        return matches

    def __construct_verdict_prompt(self, statement_list, context):
        prompt = (
            f"Consider the given context and following\n"
            f"statements, then determine whether they\n"
            f"are supported by the information present\n"
            f"in the context. Provide a brief explanation for each statement before arriving\n"
            f"at the verdict (Yes/No). Provide a final\n"
            f"verdict for each statement in order at the\n"
            f"end in the given format. Do not deviate\n"
            f"from the specified format.\n\n"
            f"Context: {context}\n\n"
        )

        #Adiciona as afirmações no prompt
        for i, statement in enumerate(statement_list, 1):
            prompt += f"Statement {i}: {statement}\n"
        #Adiciona o formato de veredito
        prompt += "\nFinal verdict format:\n"
        for i in range(1, len(statement_list) + 1):
            prompt += f"Verdict {i}: Yes/No\n"

        return prompt

    def __process_verdicts(self, verdicts):
        """
        Função para processar as respostas com vereditos
        """
        #Expressão regular para capturar vereditos
        verdict_pattern = re.findall(r"Verdict \d+:\s*(Yes|No)", verdicts)

        #Converte vereditos em 1 e 0
        binary_verdicts = [1 if v == 'Yes' else 0 for v in verdict_pattern]

        if not binary_verdicts:
            print(f"No verdicts found in the response: {verdicts}")
            return 0

        return sum(binary_verdicts) / len(binary_verdicts)

    def evaluate(self, question, answer, context):
        #Primeiro prompt para pegar as afirmações
        get_statements_prompt = f"""
        Given a question and answer, create one
        or more statements from each sentence
        in the given answer.

        Return the response in the following format:

        Statement 1: statement_1
        ...
        Statement n: statement_n

        Do not deviate from the specified format.

        question: {question}
        answer: {answer}
        """
        #Uso do modelo para capturar os statements
        statements = self.model.get_answer(get_statements_prompt)
        #Processametno das saidas
        processed_statements = self.__process_statements(statements)

        #Construir o segundo prompt
        faithfulness_prompt = self.__construct_verdict_prompt(processed_statements, context)
        #Vereditos da LLM
        verdicts = self.model.get_answer(faithfulness_prompt)

        average_score = self.__process_verdicts(verdicts)

        return average_score



class AnswerRelevance(Metric):
    def __init__(self, model):
        super().__init__(model)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

    def evaluate(self, question, answer, n=5):

        # Gera N questões a partir da resposta
        generated_questions = self.__generate_questions_from_answer(answer, n)

        #Calcula os emebeddings
        original_question_embedding = self.embedding_model.encode(question, convert_to_tensor=True)
        generated_question_embeddings = self.embedding_model.encode(generated_questions, convert_to_tensor=True)

        #Calcula similaridade por cosseno
        similarities = self.__calculate_similarities__(original_question_embedding, generated_question_embeddings)

        #Média das similaridades
        average_relevance_score = torch.mean(torch.tensor(similarities)).item()

        return average_relevance_score

    def __generate_questions_from_answer(self, answer, n):
        generated_questions = []
        for _ in range(n):
            #Prompt de geração das perguntas
            generate_question_prompt = f"Generate a question for the given answer.\nanswer: {answer}"
            question = self.model.get_answer(generate_question_prompt)
            generated_questions.append(question.strip())

        return generated_questions

    def __calculate_similarities__(self, original_embedding, generated_embeddings):

        original_embedding = original_embedding.unsqueeze(0)

        similarities = F.cosine_similarity(generated_embeddings, original_embedding, dim=1).tolist()

        return similarities

class ContextRelevance(Metric):
    def __init__(self, model):
        super().__init__(model)

    def evaluate(self, question, context):
        # Calculate total sentences across all documents in the context
        total_sentences = sum(len(self.__split_into_sentences(doc)) for doc in context)

        # Extract relevant sentences from the context
        extracted_sentences = self.__extract_relevant_sentences(question, context)

        # Calculate relevance score as the ratio of relevant sentences to total sentences
        if "Insufficient Information" in extracted_sentences:
            relevance_score = 0.0
        else:
            relevance_score = len(extracted_sentences) / total_sentences

        return relevance_score, extracted_sentences

    def __extract_relevant_sentences(self, question, context):
        # Join all documents into a single text for extraction
        context_text = " ".join(context)

        # Prompt for sentence extraction
        extract_prompt = f"""
        Please extract relevant sentences from the provided context that can potentially help answer the following question.
        If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phrase "Insufficient Information".
        While extracting candidate sentences, you’re not allowed to make any changes to sentences from the given context.

        question: {question}
        context: {context_text}
        """

        extracted_sentences = self.model.get_answer(extract_prompt)

        if "Insufficient Information" in extracted_sentences:
            return ["Insufficient Information"]
        else:
            # Process extracted sentences into a list
            return self.__process_extracted_sentences(extracted_sentences)

    def __process_extracted_sentences(self, extracted_sentences):
        # Split extracted text into sentences, stripping whitespace
        list_of_sentences = [sentence.strip() for sentence in extracted_sentences.split('. ') if sentence]

        return list_of_sentences

    def __split_into_sentences(self, document):
        return [sentence.strip() for sentence in document.split('. ') if sentence]




## Multiagents

In [ ]:
!pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 5.8 MB/s eta 0:00:00


In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END


In [13]:
from typing import TypedDict, Optional

class GraphState(TypedDict):
    original_question: Optional[str] =  None,
    options: Optional[str] = None,
    rephrased_question: Optional[str] =  None,
    rag_information: Optional[str] = None,
    cot_output: Optional[str] = None,
    faith_score: Optional[float] = None

In [8]:
def rephraser_agent(state):
    llm_caller = GPT()

    query_prompt = """Rephrase the question using medical terminology to abstract the patient's specific symptoms and conditions
Question: {}"""

    original_question = state.get('original_question', '').strip()
    rephrased_question = state.get('rephrased_question', '').strip()
    options = state.get('options', '').strip()
    rag_information = state.get('rag_information', '').strip()
    cot_output = state.get('cot_output', '')
    faith_score = state.get('faith_score', '')

    rephrased_question = llm_caller.get_answer(query_prompt.format(original_question))


    return {
            'original_question' : original_question,
            'rephrased_question' : rephrased_question,
            'options': options,
            'rag_information' : rag_information,
            'cot_output' : cot_output,
            'faith_score' : faith_score
    }

In [9]:
def cot_agent(state):
    llm_caller = GPT()

    query_prompt = """Your role is to be a medical assistant for performing Q&A
in the medical domain. Given the following question and options, return the
correct option thinking step-by-step on how to get to the final answer.
Return the answer in the following format:
Answer: (option from A to E)"""


    original_question = state.get('original_question', '').strip()
    rephrased_question = state.get('rephrased_question', '').strip()
    options = state.get('options', '').strip()
    rag_information = state.get('rag_information', '').strip()
    cot_output = state.get('cot_output', '')
    faith_score = state.get('faith_score', '')


    question = rephrased_question if rephrased_question != '' else original_question

    if rag_information != ''.strip():
      query_prompt += """\nUse the following context to answer the question: {}
Question: {}
Options:{}
      """.format(rag_information, question, options)
    else:
      query_prompt += """
Question: {}
Options:{}
      """.format(question, options)

    cot_output = llm_caller.get_answer(query_prompt)

    return {
            'original_question' : original_question,
            'rephrased_question' : rephrased_question,
            'options': options,
            'rag_information' : rag_information,
            'cot_output' : cot_output,
            'faith_score' : faith_score
    }



In [10]:
def rag_agent(state):
    llm_caller = GPT()

    relevant_prompt = """Given the following question and documents, extract only the documents that are relevant to the question
Question: {}
Documents: {}
    """

    useful_prompt = """Given the follow extract only the segments that are useful to the question
Question: {}
Documents: {}
"""


    original_question = state.get('original_question', '').strip()
    rephrased_question = state.get('rephrased_question', '').strip()
    options = state.get('options', '').strip()
    rag_information = state.get('rag_information', '').strip()
    cot_output = state.get('cot_output', '')
    answer_idx = state.get('answer_idx', '')
    faith_score = state.get('faith_score', '')

    rag_docs = rag_module.retrieve_documents(original_question, num_docs = 5)

    rag_information = "\n".join(str(item) for item in rag_docs)

    relevant = llm_caller.get_answer(relevant_prompt.format(original_question,
                                                                   rag_information))

    context = llm_caller.get_answer(useful_prompt.format(original_question,
                                                    relevant))
    return {
            'original_question' : original_question,
            'rephrased_question' : rephrased_question,
            'options': options,
            'rag_information' : context,
            'cot_output' : cot_output,
            'answer_idx' : answer_idx,
            'faith_score' : faith_score

    }

In [11]:
def faithfulness_agent(state):
    faithfulness_llm = Faithulness(GPT())

    original_question = state.get('original_question', '').strip()
    rephrased_question = state.get('rephrased_question', '').strip()
    options = state.get('options', '').strip()
    rag_information = state.get('rag_information', '').strip()
    answer = state.get('answer', '')
    faith_score = state.get('faith_score', '')

    question_plus_options = f'{original_question}\n{options}'
    faith_score = faithfulness_llm.evaluate(question_plus_options, answer, rag_information)

    return {
            'original_question' : original_question,
            'rephrased_question' : rephrased_question,
            'options': options,
            'rag_information' : rag_information,
            'answer' : answer,
            'faith_score' : faith_score

    }

In [ ]:
medqa_workflow = StateGraph(GraphState)

medqa_workflow.add_node("rephraser_agent", rephraser_agent)
medqa_workflow.add_node("cot_agent", cot_agent)
medqa_workflow.add_node("rag_agent", rag_agent)
medqa_workflow.add_node("faithfulness_agent", faithfulness_agent)

medqa_workflow.set_entry_point("rephraser_agent")
medqa_workflow.add_edge("rephraser_agent","rag_agent")
medqa_workflow.add_edge("rag_agent","cot_agent")
medqa_workflow.add_edge("cot_agent","faithfulness_agent")
medqa_workflow.add_edge("faithfulness_agent",END)


In [ ]:
state = {
    'original_question' : train_subset[0]['question'],
    'rephrased_question' : '',
    'options':str(train_subset[0]['options']),
    'rag_information' : '',
    'cot_output' : '',
    'faith_score': 0
}

In [ ]:
app = medqa_workflow.compile()
conversation = app.invoke(state)

In [ ]:
conversation

## Strategy Testing functions

In [66]:
def test_model_without_agents(test_set,llm):
    query_prompt = """Given the following question and options, return the answer in the following format:
    Answer: (option from A to E)

    Question: {}
    Options:{}"""

    test_results = {'questions':[],
                    'options':[],
                    'answer_idx': [],
                    'llm_answer': []}

    for item in test_set:
        question = item['question']
        options = item['options']
        answer_idx = item['answer_idx']



        llm_answer = llm.get_answer(query_prompt.format(question, options))
        print(query_prompt.format(question, options))
        print(llm_answer)
        test_results['questions'].append(question)
        test_results['options'].append(options)
        test_results['answer_idx'].append(answer_idx)
        test_results['llm_answer'].append(llm_answer.split('Answer:')[1].strip())

    return test_results

In [ ]:

from tqdm import tqdm
def load_checkpoint(filepath):
    """Loads checkpoint data from CSV and returns it as a dictionary."""
    df = pd.read_csv(filepath)
    return {
        'questions': df['questions'].tolist(),
        'options': df['options'].tolist(),
        'answer_idx': df['answer_idx'].tolist(),
        'llm_answer': df['llm_answer'].tolist(),
        'cot_output': df['cot_output'].tolist(),
        'faithfulness': df['faithfulness'].tolist(),
        'rag_information': df['rag_information'].tolist()
    }, len(df)


def test_workflow(workflow, test_set, test_type='all', checkpoint_path = None):
    # Load last checkpoint if available
    if checkpoint_path is not None:
        print(f"Resuming from checkpoint: {checkpoint_path}")
        test_results, start_index = load_checkpoint(checkpoint_path)

    else:
        print("Starting a new experiment.")
        test_results = {
            'questions': [],
            'options': [],
            'answer_idx': [],
            'llm_answer': [],
            'cot_output': [],
            'faithfulness': [],
            'rag_information': []
        }
        start_index = 0

    app = workflow.compile()
    checkpoint_interval = max(1, len(test_set) // 10)  # Calculate 10% interval

    # Continue from the last processed sample
    for i, item in enumerate(tqdm(test_set[start_index:], desc="Testing workflow"), start=start_index + 1):
        question = item['question']
        options = item['options']
        answer_idx = item['answer_idx']
        state = {
            'original_question': question,
            'rephrased_question': '',
            'options': str(options),
            'rag_information': '',
            'cot_output': '',
            'faith_score': 0
        }

        conversation = app.invoke(state)
        llm_answer = conversation['cot_output'].split('Answer:')[-1].strip()

        test_results['questions'].append(question)
        test_results['options'].append(options)
        test_results['answer_idx'].append(answer_idx)
        test_results['llm_answer'].append(llm_answer)
        test_results['cot_output'].append(conversation['cot_output'])
        test_results['rag_information'].append(conversation.get('rag_information', ''))
        test_results['faithfulness'].append(conversation['faith_score'])

        # Save checkpoint every 10% of the process
        if i % checkpoint_interval == 0:
            df = pd.DataFrame(test_results)
            df.to_csv(f'test_results_checkpoint_{test_type}_{i // checkpoint_interval}.csv', index=False)

    df = pd.DataFrame(test_results)
    df.to_csv(f'test_results_final_{test_type}.csv', index=False)

    return test_results

# Tests

## Testing Chunking Strategies

In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00


In [4]:
model_name = "sentence-transformers/all-mpnet-base-v2"
path = 'medical_textbooks/'
rag_module = RAG(model_name, path)

/tmp/ipykernel_1065/520273307.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedding_model = HuggingFaceEmbeddings(


In [ ]:
dataset_path = 'data_clean/'

rag_module.create_vectorstore(dataset_path)

In [ ]:
query = "What substances are nonresearch use illegal under federal law?"

rag_module.retrieve_documents(query, 2)

['(All nonresearch use illegal under federal law.)\n\n\n\nFlunitrazepam (Rohypnol) Narcotics:\n\n\n\nHallucinogens:\n\n\n\nLSD MDA, STP, DMT, DET, mescaline, peyote, bufotenine, ibogaine, psilocybin, phencyclidine (PCP; veterinary drug only) (No telephone prescriptions, no refills.)2\n\n\n\nOpioids: Opium: Opium alkaloids and derived phenanthrene alkaloids: codeine, morphine (Avinza, Kadian, MSContin, Roxanol), hydrocodone and hydrocodone combinations (Zohydro ER, Hycodan, Vicodin, Lortab), hydromorphone (Dilaudid), oxymorphone (Exalgo), oxycodone (dihydroxycodeinone, a component of Oxycontin, Percodan, Percocet, Roxicodone, Tylox)\n\n\n\nDesignated synthetic drugs: meperidine (Demerol), methadone, levorphanol (Levo-Dromoran), fentanyl (Duragesic, Actiq, Fentora), alfentanil (Alfenta), sufentanil (Sufenta), remifentanil (Ultiva), tapentadol (Nycynta)\n\n\n\nStimulants: Coca leaves and cocaine Amphetamines: Amphetamine complex (Biphetamine), Amphetamine salts (Adderall), Dextroamphetami

In [ ]:

model_name = "sentence-transformers/all-mpnet-base-v2"
path_recursive = 'medical_textbooks_recursive/'
rag_module_recursive = RAG(model_name, path_recursive)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
dataset_path = 'data_clean/'
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
rag_module_recursive.create_vectorstore(dataset_path, text_splitter = text_splitter)

In [ ]:
query = "What substances are nonresearch use illegal under federal law?"

rag_module_recursive.retrieve_documents(query, 2)

['(All nonresearch use illegal under federal law.)\n\n\n\nFlunitrazepam (Rohypnol) Narcotics:\n\n\n\nHallucinogens:\n\n\n\nLSD MDA, STP, DMT, DET, mescaline, peyote, bufotenine, ibogaine, psilocybin, phencyclidine (PCP; veterinary drug only) (No telephone prescriptions, no refills.)2\n\n\n\nOpioids: Opium: Opium alkaloids and derived phenanthrene alkaloids: codeine, morphine (Avinza, Kadian, MSContin, Roxanol), hydrocodone and hydrocodone combinations (Zohydro ER, Hycodan, Vicodin, Lortab), hydromorphone (Dilaudid), oxymorphone (Exalgo), oxycodone (dihydroxycodeinone, a component of Oxycontin, Percodan, Percocet, Roxicodone, Tylox)\n\n\n\nDesignated synthetic drugs: meperidine (Demerol), methadone, levorphanol (Levo-Dromoran), fentanyl (Duragesic, Actiq, Fentora), alfentanil (Alfenta), sufentanil (Sufenta), remifentanil (Ultiva), tapentadol (Nycynta)',
 'Stimulants:\n\n\n\nDepressants:\n\n\n\nSchedule II barbiturates in mixtures with noncontrolled drugs or in suppository dosage form Ba

In [ ]:
context_relevance_metric = ContextRelevance(gpt4o)

In [ ]:
#Criando subset de treino
path_questions ='data_clean/questions/US/train.jsonl'
with open(path_questions, 'r') as file:
    train_set = [json.loads(line) for line in file]
train_set[0]

import random
random.seed(15)
random.shuffle(train_set)


train_subset = train_set[:200]

In [ ]:
train_subset

[{'question': 'A 75-year-old man is brought to the emergency department for a 5-day-history of worsening dyspnea, orthopnea, and lower leg swelling. He has a history of hypertension, hyperlipidemia, non-alcoholic fatty liver disease, and myocardial infarction 10 years ago. Current medications include metoprolol, lisinopril, ethacrynic acid, eplerenone, and aspirin. He drinks 1 beer daily. He has a 30-pack-year smoking history. He is allergic to sulfonamides. His temperature is 37.0°C (98.6°F), his pulse is 120/min, and his blood pressure is 120/80 mm Hg. Physical examination reveals jugular venous distention and 3+ pitting edema in his lower legs. Crackles are heard at both lung bases. The point of maximal impulse is 2 cm to the left of the midclavicular line in the 6th intercostal space. Which of the following additional findings would be most strongly associated with increased mortality?',
  'answer': 'Decreased serum Na+',
  'options': {'A': 'Decreased BNP levels',
   'B': 'Decrease

In [ ]:
def convert_options_to_string(d):
    result = ""
    for key, value in d.items():
        if value:
            result += f"{key}:{value}\n"
        else:
            result += f"{key}:\n"
    return result


In [ ]:
train_subset[0]

{'question': 'A 75-year-old man is brought to the emergency department for a 5-day-history of worsening dyspnea, orthopnea, and lower leg swelling. He has a history of hypertension, hyperlipidemia, non-alcoholic fatty liver disease, and myocardial infarction 10 years ago. Current medications include metoprolol, lisinopril, ethacrynic acid, eplerenone, and aspirin. He drinks 1 beer daily. He has a 30-pack-year smoking history. He is allergic to sulfonamides. His temperature is 37.0°C (98.6°F), his pulse is 120/min, and his blood pressure is 120/80 mm Hg. Physical examination reveals jugular venous distention and 3+ pitting edema in his lower legs. Crackles are heard at both lung bases. The point of maximal impulse is 2 cm to the left of the midclavicular line in the 6th intercostal space. Which of the following additional findings would be most strongly associated with increased mortality?',
 'answer': 'Decreased serum Na+',
 'options': {'A': 'Decreased BNP levels',
  'B': 'Decreased se

In [ ]:
train_subset[0]['question'] + '\n'+ convert_options_to_string(train_subset[0]['options'])

'A 75-year-old man is brought to the emergency department for a 5-day-history of worsening dyspnea, orthopnea, and lower leg swelling. He has a history of hypertension, hyperlipidemia, non-alcoholic fatty liver disease, and myocardial infarction 10 years ago. Current medications include metoprolol, lisinopril, ethacrynic acid, eplerenone, and aspirin. He drinks 1 beer daily. He has a 30-pack-year smoking history. He is allergic to sulfonamides. His temperature is 37.0°C (98.6°F), his pulse is 120/min, and his blood pressure is 120/80 mm Hg. Physical examination reveals jugular venous distention and 3+ pitting edema in his lower legs. Crackles are heard at both lung bases. The point of maximal impulse is 2 cm to the left of the midclavicular line in the 6th intercostal space. Which of the following additional findings would be most strongly associated with increased mortality?\nA:Decreased BNP levels\nB:Decreased serum Na+\nC:Decreased QRS complex duration\nD:Increased VO2\nE:Increased 

In [ ]:
train_set_results = {
    'questions': [],
    'contexts_semantic': [],
    'extracted_sentences_semantic':[],
    'context_relevance_semantic':[],
    'contexts_recursive':[],
    'extracted_sentences_recursive':[],
    'context_relevance_recursive':[],
}

for item in train_subset:
    question = f'{item["question"]}\n{convert_options_to_string(item["options"])}'

    #Avaliando retriever semantico
    context_semantic = rag_module.retrieve_documents(question, num_docs = 2)
    context_relevance_semantic, extracted_sentences_semantic = context_relevance_metric.evaluate(question, context_semantic)

    #Avaliando Retriever recursivo
    context_recursive = rag_module_recursive.retrieve_documents(question, num_docs = 2)
    context_relevance_recursive,  extracted_sentences_recursive = context_relevance_metric.evaluate(question, context_recursive)

    train_set_results['questions'].append(question)

    train_set_results['contexts_semantic'].append(context_semantic)
    train_set_results['extracted_sentences_semantic'].append(extracted_sentences_semantic)
    train_set_results['context_relevance_semantic'].append(context_relevance_semantic)

    train_set_results['contexts_recursive'].append(context_recursive)
    train_set_results['extracted_sentences_recursive'].append(extracted_sentences_recursive)
    train_set_results['context_relevance_recursive'].append(context_relevance_recursive)


In [ ]:
results_df = pd.DataFrame.from_dict(train_set_results)
results_df

,questions,contexts_semantic,extracted_sentences_semantic,context_relevance_semantic,contexts_recursive,extracted_sentences_recursive,context_relevance_recursive
0,A 75-year-old man is brought to the emergency ...,[The pain is accompanied by dyspnea (shortness...,[Insufficient Information],0.000000,[T AB LE 2.1 -2. Supraventricular Tachyarrhyt...,[Insufficient Information],0.00
1,A 72-year-old man with a 4-year history of Par...,[Central Nervous System Disorders 1. Parkinson...,"[1, ""With continued treatment, however, the du...",0.093750,"[When symptomatic treatment becomes necessary,...",[Insufficient Information],0.00
2,A 5-week-old male infant is brought to the Eme...,[The chest should be auscultated for evidence ...,"[1, ""Pyloric stenosis occurs in the first mont...",0.178571,"[In neonates with true vomiting, congenital ob...","[""Pyloric stenosis occurs in the first months ...",0.10
3,A 65-year-old man presents to his primary care...,[479e. Diseases with fever and rash may be cla...,[Insufficient Information],0.000000,"[The age of the patient, onset, duration, prog...",[Insufficient Information],0.00
4,A 60-year-old woman is brought to the emergenc...,[One of our patients had intermittent seizures...,[Insufficient Information],0.000000,[Cerebral edema is the accumulation of excess ...,[Insufficient Information],0.00
...,...,...,...,...,...,...,...
195,A 33-year-old female presents with recent onse...,[D. May slowly progress to renal failure\n\n\n...,"[1, ""Infection of the kidney""\n2, ""Presents wi...",0.095238,"[Culture-greater than 100,000 colony forming u...","[1, ""A, Infection of the kidney 1, Usually due...",0.20
196,A 25-year-old man presents with a nodule on hi...,[Subcutaneous nodular lesions have also been i...,[Insufficient Information],0.000000,[Hepatosplenomegaly and lymphadenopathy are ea...,[Insufficient Information],0.00
197,A molecular biologist is studying the roles of...,[The\n\n\n\nNav1.5 SCN5A\n\n\n\nCav1.2 CACNA1C...,[Insufficient Information],0.000000,[0 1.5 3 4.5 10 msec pA Channel #1 current Cha...,"[During the action potential plateau, Ca++ ent...",0.25
198,A 37-year-old obese woman presents to the neur...,[A 35-year-old woman comes to her physician co...,[Insufficient Information],0.000000,[syndrome is commonly associated with pregnanc...,[Insufficient Information],0.00


In [ ]:
results_df['questions'].iloc[0]

'A 75-year-old man is brought to the emergency department for a 5-day-history of worsening dyspnea, orthopnea, and lower leg swelling. He has a history of hypertension, hyperlipidemia, non-alcoholic fatty liver disease, and myocardial infarction 10 years ago. Current medications include metoprolol, lisinopril, ethacrynic acid, eplerenone, and aspirin. He drinks 1 beer daily. He has a 30-pack-year smoking history. He is allergic to sulfonamides. His temperature is 37.0°C (98.6°F), his pulse is 120/min, and his blood pressure is 120/80 mm Hg. Physical examination reveals jugular venous distention and 3+ pitting edema in his lower legs. Crackles are heard at both lung bases. The point of maximal impulse is 2 cm to the left of the midclavicular line in the 6th intercostal space. Which of the following additional findings would be most strongly associated with increased mortality?\nA:Decreased BNP levels\nB:Decreased serum Na+\nC:Decreased QRS complex duration\nD:Increased VO2\nE:Increased 

In [ ]:
results_df['contexts_semantic'].iloc[0]

['The pain is accompanied by dyspnea (shortness of breath), diaphoresis (sweating), and nausea. Focused History: BJ reports episodes of exertional chest pain in the last few months, but they were less severe and of short duration. He smokes (2–3 packs per day), drinks alcohol only rarely, eats a “typical” diet, and walks with his wife most weekends. His blood pressure has been normal. Family history reveals that his father and paternal aunt died of heart disease at age 45 and 39 years, respectively. His mother and younger (age 31 years) brother are said to be in good health. Physical Examination (Pertinent Findings): BJ is pale and clammy and is in distress due to chest pain. Blood pressure and respiratory rate are elevated. Lipid deposits are noted on the periphery of his corneas (corneal arcus; see left image) and under the skin on and around his eyelids (xanthelasmas; see right image). No deposits on his tendons (xanthomas) are detected. Pertinent Test Results: BJ’s electrocardiogra

In [ ]:
results_df['contexts_recursive'].iloc[0]

['T AB LE 2.1 -2.  Supraventricular Tachyarrhythmias\n\n\n\nAtrial Sinus tachycardia Normal physiologic response to fear, pain, and exercise. Can also be 2° to hyperthyroidism, volume contraction, infection, or pulmonary embolism. Palpitations, shortness of breath. Ventricular rate > 100 bpm; normal P waves before every QRS complex. Treat the underlying cause. Atrial f brillation (AF) Acute AF—   PIRATES: Pulmonary disease Ischemia Rheumatic heart disease Anemia/Atrial myxoma Thyrotoxicosis Ethanol Sepsis Chronic AF— hypertension, CHF. Often asymptomatic, but may present with shortness of breath, chest pain, or palpitations. Physical exam reveals irregularly irregular pulse. No discernible P waves, with variable and irregular QRS response.\n\n\n\nEstimate risk of stroke using CHAD2 score.\n\n\n\nAnticoagulation if > 48 hours (to prevent CVA); rate control (CCBs, β-blockers, digoxin, amiodarone).',
 'PART 2 Cardinal Manifestations and Presentation of Diseases\n\n\n\nVital Signs Signific

In [ ]:
results_df.to_csv('retriever_context_relevance.csv', index = None)

In [ ]:
results_df.head(15)

,questions,contexts_semantic,extracted_sentences_semantic,context_relevance_semantic,contexts_recursive,extracted_sentences_recursive,context_relevance_recursive
0,A 75-year-old man is brought to the emergency ...,[The pain is accompanied by dyspnea (shortness...,[Insufficient Information],0.000000,[T AB LE 2.1 -2. Supraventricular Tachyarrhyt...,[Insufficient Information],0.000000
1,A 72-year-old man with a 4-year history of Par...,[Central Nervous System Disorders 1. Parkinson...,"[1, ""With continued treatment, however, the du...",0.093750,"[When symptomatic treatment becomes necessary,...",[Insufficient Information],0.000000
2,A 5-week-old male infant is brought to the Eme...,[The chest should be auscultated for evidence ...,"[1, ""Pyloric stenosis occurs in the first mont...",0.178571,"[In neonates with true vomiting, congenital ob...","[""Pyloric stenosis occurs in the first months ...",0.100000
3,A 65-year-old man presents to his primary care...,[479e. Diseases with fever and rash may be cla...,[Insufficient Information],0.000000,"[The age of the patient, onset, duration, prog...",[Insufficient Information],0.000000
4,A 60-year-old woman is brought to the emergenc...,[One of our patients had intermittent seizures...,[Insufficient Information],0.000000,[Cerebral edema is the accumulation of excess ...,[Insufficient Information],0.000000
5,A 26-year-old male is brought into the emergen...,[Was MW in positive or negative nitrogen balan...,[Insufficient Information],0.000000,[The plasma Na+ concentration on admission was...,[Insufficient Information],0.000000
6,"A 31-year-old male with cirrhosis, dementia, a...",[■↓ ceruloplasmin and excessive deposition of ...,[■↓ ceruloplasmin and excessive deposition of ...,0.041096,[Correct answer = H. The patient has Wilson di...,"[""The patient has Wilson disease, an autosomal...",0.090909
7,"An experimental drug, ES 62, is being studied....","[−K\n\n\n\nThe higher the Km, the lower the af...",[Insufficient Information],0.000000,[C. Volume of Distribution\n\n\n\nThe apparent...,[The volume of distribution is the proportiona...,0.125000
8,Patient 1 – A 26-year-old woman presents to he...,"[Dunnwald LK, Rossing MA, Li CI. Hormone recep...",[Insufficient Information],0.000000,[If the presence of breast cancer is strongly ...,[Insufficient Information],0.000000
9,"A 45-year-old woman, gravida 3, para 2, at 18 ...",[1 107\n\n\n\nSCREENING AND DIAGNOSIS ...........,[Insufficient Information],0.000000,"[Rood K, Markham KB: Torsion of a term gravid ...",[Insufficient Information],0.000000


In [ ]:
results_df['context_relevance_recursive'].mean()

0.0367118691851664

In [ ]:
results_df['context_relevance_semantic'].mean()

0.015717990033766043

## Creating test set and subset

In [17]:


path_questions ='data_clean/questions/US/test.jsonl'
with open(path_questions, 'r') as file:
    test_set = [json.loads(line) for line in file]
test_set[0]

import random
random.seed(15)
random.shuffle(test_set)


test_subset = test_set[:50]

In [ ]:
test_set[0]

{'question': 'A 32-year-old man with HIV comes to the physician because of a 2-month history of weight loss, night sweats, and productive cough. Auscultation of the lungs shows coarse crackles at the right upper posterior field. An x-ray of the chest shows an opacity in the right upper lobe. Sputum analysis shows acid-fast bacilli. A small amount of tuberculin fluid is injected into the subcutaneous tissue on the left forearm. Examination of the injected area 48 hours later shows no induration or erythema. Impairment of which of the following processes is the most likely cause of the negative tuberculin skin test seen in this patient?',
 'answer': 'Interaction of B7 and CD28 ligands',
 'options': {'A': 'Secretion of interferon-α',
  'B': 'Interaction of B7 and CD28 ligands',
  'C': 'Opsonization by complement proteins',
  'D': 'Secretion of interleukin-4',
  'E': 'Generation of reactive oxygen species'},
 'meta_info': 'step1',
 'answer_idx': 'B'}

## Testing GPT4o-mini without agents

In [67]:
llm = GPT()
results_without_agents = test_model_without_agents(test_set, llm)

Given the following question and options, return the answer in the following format:
    Answer: (option from A to E)

    Question: An investigator is studying the effects of different gastrointestinal regulatory substances. A healthy subject is asked to eat a meal at hour 0, and the pH of stomach contents and rate of stomach acid secretions are measured over the next 4 hours. Results of the study are shown. Which of the following mechanisms most likely contributes to the changes seen at point D in the graph?
    Options:{'A': 'Increased vagal stimulation', 'B': 'Increased activity of D cells', 'C': 'Increased activation of H2 receptors', 'D': 'Increased activity of enterochromaffin-like cells', 'E': 'Increased activity of I cells'}
To determine the mechanism contributing to the changes seen at point D in the study of gastrointestinal regulatory substances, we need to consider the physiological responses to a meal and the roles of the various cells and mechanisms listed in the options

In [ ]:
sum([results_without_agents['answer_idx'][i] == results_without_agents['llm_answer'][i] for i in range(len(results_without_agents['questions']))])/len(results_without_agents['questions'])

0.6897093479968578

In [ ]:
results_without_agents_df = pd.DataFrame.from_dict(results_without_agents)
results_without_agents_df.to_csv('results_without_agents.csv', index = None)

## Testing CoT Agent

In [ ]:
re_write_workflow = StateGraph(GraphState)

re_write_workflow.add_node("cot_agent", cot_agent)


re_write_workflow.set_entry_point("cot_agent")
re_write_workflow.add_edge("cot_agent",END)



In [ ]:
results_with_cot = test_workflow(re_write_workflow, test_set)

A saída de streaming foi truncada nas últimas 5000 linhas.

Let's analyze the options given:

- **A: Conjugation** - This is a process where bacteria transfer genetic material through direct cell-to-cell contact. It typically requires a conjugative plasmid and is not indicated as directly relevant in this scenario as there is no mention of direct cell interaction.

- **B: Reassortment** - This is a process seen typically in viruses, particularly segmented viruses. It does not apply to bacterial exchanges.

- **C: Transformation** - This process involves a bacterium taking up free DNA from its environment (which is what would happen with the lysed bacteria). This aligns with the scenario since the noninfectious bacteria could take up virulence genes from the infectious bacteria's lysed contents.

- **D: Generalized transduction** - This process involves transferring of DNA from one bacterium to another via a bacteriophage. There's no mention of phages in the scenario, thus it doesn't fi

In [ ]:
sum([results_with_cot['answer_idx'][i] == results_with_cot['llm_answer'][i] for i in range(len(results_with_cot['questions']))])/len(results_with_cot['questions'])

0.7195600942655145

In [ ]:
results_with_cot_df = pd.DataFrame.from_dict(results_with_cot)
results_with_cot_df.to_csv('results_with_cot_all.csv', index = None)

## Testing with Rewrite + CoT agents

In [ ]:
re_write_workflow = StateGraph(GraphState)

re_write_workflow.add_node("rephraser_agent", rephraser_agent)
re_write_workflow.add_node("cot_agent", cot_agent)


re_write_workflow.set_entry_point("rephraser_agent")
re_write_workflow.add_edge("rephraser_agent","cot_agent")
re_write_workflow.add_edge("cot_agent",END)



In [ ]:
results_with_rephrasal = test_workflow(re_write_workflow, test_set)

-----CoT Prompt-----
Your role is to be a medical assistant for performing Q&A
in the medical domain. Given the following question and options, return the
correct option thinking step-by-step on how to get to the final answer.
Return the answer in the following format:
Answer: (option from A to E)
Question: A 64-year-old male presents to his primary care provider with a chief complaint of bilateral knee arthralgia, characterized by exacerbation over several years, particularly noted during activities such as stair ascent and ambulation exceeding 100 yards. The patient describes the nociception intensifying with physical exertion and experiencing symptomatic relief via rest periods. He reports minimal matutinal stiffness lasting approximately 5-10 minutes upon awakening. The physical examination reveals tenderness on palpation of the medial tibiofemoral compartment of both knees, alongside crepitus and a limited range of motion at the extremes of flexion and extension. Both knee joints 

Testing workflow:  98%|█████████████████████████████████████████████████████████▋ | 1244/1273 [3:59:28<08:12, 16.98s/it]

-----CoT Output-------
To determine the correct option for further evaluation of the condition presented in the scenario, let’s analyze the symptoms and findings.

1. **Patient's Age and Symptoms**: The patient is a 64-year-old male with bilateral knee arthralgia that has worsened over several years. Symptoms are aggravated by physical activity and relieved by rest, which indicates a potential degenerative process rather than inflammatory arthritis.

2. **Morning Stiffness**: The patient reports only minimal morning stiffness lasting around 5-10 minutes, which is not typical for inflammatory arthritis conditions that usually present with longer periods of stiffness.

3. **Physical Examination Findings**: 
   - Tenderness primarily in the medial tibiofemoral compartment suggests an inner knee issue.
   - Presence of crepitus and limited range of motion at extremes of knee movement suggest degenerative changes.
   - Coolness of joints and bony enlargement at the medial joint line points 

Testing workflow:  98%|█████████████████████████████████████████████████████████▋ | 1245/1273 [3:59:39<07:06, 15.25s/it]

-----CoT Output-------
To determine the correct physiological consequence of the observed impaired renal perfusion in this patient, we first need to analyze the situation step-by-step:

1. **Patient Background**: The patient has a history of cardiovascular issues and high blood pressure, and his blood pressure is currently poorly controlled despite pharmacotherapy.

2. **Findings**: The renal duplex ultrasonography identifies a 90% stenosis of the right renal artery. This indicates a significant reduction in blood flow to this kidney, leading to impaired renal perfusion.

3. **Renal Physiology**: The kidneys regulate blood pressure through several mechanisms, primarily involving the renin-angiotensin-aldosterone system (RAAS). Impaired renal perfusion typically leads to activation of this system.

4. **Juxtaglomerular Apparatus**: The juxtaglomerular cells, found in the kidney, act in response to low perfusion pressure by secreting renin. Renin secretion increases in response to low re

...

-----CoT Output-------
To determine the correct answer, let's analyze the scenario and the implications of a third-degree perineal laceration:

1. **Understanding the Laceration**: A third-degree perineal laceration extends through the vaginal mucosa, perineal body, and into the anal sphincter complex. This type of injury can significantly disrupt the function of the anal sphincter, which is crucial for maintaining fecal continence.

2. **Potential Complications**: When considering complications following delivery with a third-degree laceration, we need to focus on how this injury can affect the woman's body:
   - **Urge incontinence (A)**: This generally relates to bladder issues, not directly caused by perineal lacerations.
   - **Vesicovaginal fistula (B)**: This is a potential complication following childbirth that involves a connection between the bladder and vagina, but it is more associated with fourth-degree lacerations and specific obstetric trauma.
   - **Fecal incontinence (

Testing workflow: 100%|██████████████████████████████████████████████████████████▉| 1272/1273 [4:05:17<00:12, 12.52s/it]

-----CoT Output-------
To determine the most probable etiological microorganism responsible for the patient's pulmonary condition, let’s analyze the provided information step-by-step.

1. **Patient Background:**
   - The patient is a 62-year-old male who recently experienced a cerebrovascular accident (stroke). He has been hospitalized for a week.

2. **Presenting Symptoms:**
   - He presents with pyrexia (fever) and a productive cough, producing purulent sputum.
   - Vital signs indicate he is stable, though he has an elevated temperature (38.4°C).
   - The presence of basal crackles localized to the right hemithorax suggests possible consolidation or fluid in the lung.

3. **Radiology Findings:**
   - Chest X-ray demonstrates new pulmonary consolidation in the right lung, which is indicative of pneumonia or at least an infectious process affecting the lung.

4. **Laboratory Analysis:**
   - Complete blood count shows leukocytosis with a predominance of neutrophils (72%), which sugges

Testing workflow: 100%|███████████████████████████████████████████████████████████| 1273/1273 [4:05:26<00:00, 11.57s/it]

-----CoT Output-------
To determine the correct option, we need to understand the mechanism of action of miglitol, which is an alpha-glucosidase inhibitor. This type of medication works by inhibiting enzymes that break down carbohydrates in the intestines, particularly those that hydrolyze glycosidic bonds in carbohydrates.

Now, let's analyze the options:
- **A: Phosphodiester bonds** - These bonds are found in nucleic acids (DNA/RNA) and are not relevant to carbohydrate digestion.
- **B: Glycosidic bonds** - These bonds are found between monosaccharides in carbohydrates and are the primary target of miglitol. Inhibiting enzymes that break down glycosidic bonds would slow down carbohydrate absorption.
- **C: Peptide bonds** - These bonds link amino acids in proteins, which are not relevant to the effects of miglitol.
- **D: Cystine bonds** - Cystine is formed by the oxidation of cysteine residues; these are not relevant in this context.
- **E: Hydrogen bonds** - While these bonds are 

In [ ]:
sum([results_with_rephrasal['answer_idx'][i] == results_with_rephrasal['llm_answer'][i] for i in range(len(results_with_rephrasal['questions']))])/len(results_with_rephrasal['questions'])

0.6716417910447762

In [ ]:
results_with_rephrasal = pd.DataFrame.from_dict(results_with_rephrasal)
results_with_rephrasal.to_csv('results_with_rephrasal_all.csv', index = None)

## Testing RAG + CoT Agents

In [ ]:
rag_workflow = StateGraph(GraphState)

rag_workflow.add_node("cot_agent", cot_agent)
rag_workflow.add_node("rag_agent", rag_agent)
rag_workflow.add_node("faithfulness_agent", faithfulness_agent)

rag_workflow.set_entry_point("rag_agent")
rag_workflow.add_edge("rag_agent","cot_agent")
rag_workflow.add_edge("cot_agent","faithfulness_agent")
rag_workflow.add_edge("faithfulness_agent",END)



In [ ]:
results_with_rag = test_workflow(rag_workflow, test_set, test_type = 'rag_all',checkpoint_path= 'test_results_checkpoint_rag_all_7.csv')

Resuming from checkpoint: test_results_checkpoint_rag_all_7.csv


Testing workflow: 100%|█████████████████████████████████████████████████████████████| 384/384 [3:33:37<00:00, 33.38s/it]


In [ ]:
sum([results_with_rag['answer_idx'][i] == results_with_rag['llm_answer'][i] for i in range(len(results_with_rag['questions']))])/len(results_with_rag['questions'])

0.6865671641791045

In [ ]:
results_with_rag_df = pd.DataFrame.from_dict(results_with_rag)
results_with_rag_df.to_csv('results_with_rag_all.csv', index = None)

## Testing Re-write + RAG + CoT Agents

In [15]:
reph_rag_workflow = StateGraph(GraphState)


reph_rag_workflow.add_node("rephraser_agent", rephraser_agent)
reph_rag_workflow.add_node("cot_agent", cot_agent)
reph_rag_workflow.add_node("rag_agent", rag_agent)
reph_rag_workflow.add_node("faithfulness_agent", faithfulness_agent)

reph_rag_workflow.set_entry_point("rephraser_agent")
reph_rag_workflow.add_edge("rephraser_agent","rag_agent")
reph_rag_workflow.add_edge("rag_agent","cot_agent")
reph_rag_workflow.add_edge("cot_agent","faithfulness_agent")
reph_rag_workflow.add_edge("faithfulness_agent",END)



In [20]:
results_reph_rag = test_workflow(reph_rag_workflow, test_set,test_type =  'reph_rag_all', checkpoint_path='test_results_checkpoint_reph_rag_all_5.csv' )


Resuming from checkpoint: test_results_checkpoint_reph_rag_all_5.csv


Testing workflow: 100%|█████████████████████████████████████████████████████████████| 638/638 [7:25:56<00:00, 41.94s/it]


In [21]:
sum([results_reph_rag['answer_idx'][i] == results_reph_rag['llm_answer'][i] for i in range(len(results_reph_rag['questions']))])/len(results_reph_rag['questions'])

0.6834249803613511

In [22]:
results_reph_rag_df = pd.DataFrame.from_dict(results_reph_rag)
results_reph_rag_df.to_csv('results_reph_rag_all.csv', index = None)

# Analysing Results

## Preliminary Results

In [95]:
without_agents_partial = pd.read_csv('results_without_agents_partial.csv')
len(without_agents_partial[without_agents_partial['answer_idx'] == without_agents_partial['llm_answer']])/len(without_agents_partial)

0.58

In [98]:
cot_partial = pd.read_csv('results_with_cot_partial.csv')
len(cot_partial[cot_partial['answer_idx'] == cot_partial['llm_answer']])/len(cot_partial)

0.62

In [100]:
rewrite_partial = pd.read_csv('results_with_rephrasal_partial.csv')
len(rewrite_partial[rewrite_partial['answer_idx'] == rewrite_partial['llm_answer']])/len(rewrite_partial)

0.64

In [101]:
rag_partial = pd.read_csv('results_with_rag_semantic_partial.csv')
len(rag_partial[rag_partial['answer_idx'] == rag_partial['llm_answer']])/len(rag_partial)

0.64

In [104]:
whole_workflow_partial = pd.read_csv('results_reph_rag_partial.csv')
len(whole_workflow_partial[whole_workflow_partial['answer_idx'] == whole_workflow_partial['llm_answer']])/len(whole_workflow_partial)

0.7

## Final Results

In [96]:
without_agents = pd.read_csv('results_without_agents.csv')
without_agents

,questions,options,answer_idx,llm_answer
0,A 32-year-old man with HIV comes to the physic...,"{'A': 'Secretion of interferon-α', 'B': 'Inter...",B,A
1,A one-day-old male is evaluated in the hospita...,"{'A': 'Duodenal atresia', 'B': 'Intestinal mal...",A,A
2,A 38-year-old woman presents to the office for...,"{'A': 'Body dysmorphic disorder', 'B': 'Schizo...",C,C
3,A 47-year-old woman is brought to the emergenc...,"{'A': 'COPD exacerbation', 'B': 'Diabetic keto...",E,E
4,A 10-year-old boy is brought to the emergency ...,{'A': 'Hypoxanthine-guanine phosphoribosyl tra...,D,C
...,...,...,...,...
1268,A 16-month-old boy is brought to the physician...,{'A': 'Deficiency of cofactor for prolyl and l...,C,C
1269,A 21-year-old man comes to the physician becau...,"{'A': 'Wood lamp examination', 'B': 'Skin cult...",C,C
1270,"A 27-year-old woman, gravida 1, para 0, at 38 ...","{'A': 'Urge incontinence', 'B': 'Vesicovaginal...",C,C
1271,A 62-year-old patient has been hospitalized fo...,"{'A': 'Pseudomona aeruginosa', 'B': 'Streptoco...",E,A


In [97]:
len(without_agents[without_agents['answer_idx'] == without_agents['llm_answer']])/len(without_agents)

0.6897093479968578

In [71]:
cot = pd.read_csv('results_with_cot_all.csv')
cot

,questions,options,answer_idx,llm_answer
0,A 32-year-old man with HIV comes to the physic...,"{'A': 'Secretion of interferon-α', 'B': 'Inter...",B,B
1,A one-day-old male is evaluated in the hospita...,"{'A': 'Duodenal atresia', 'B': 'Intestinal mal...",A,A
2,A 38-year-old woman presents to the office for...,"{'A': 'Body dysmorphic disorder', 'B': 'Schizo...",C,C
3,A 47-year-old woman is brought to the emergenc...,"{'A': 'COPD exacerbation', 'B': 'Diabetic keto...",E,E
4,A 10-year-old boy is brought to the emergency ...,{'A': 'Hypoxanthine-guanine phosphoribosyl tra...,D,D
...,...,...,...,...
1268,A 16-month-old boy is brought to the physician...,{'A': 'Deficiency of cofactor for prolyl and l...,C,C
1269,A 21-year-old man comes to the physician becau...,"{'A': 'Wood lamp examination', 'B': 'Skin cult...",C,C
1270,"A 27-year-old woman, gravida 1, para 0, at 38 ...","{'A': 'Urge incontinence', 'B': 'Vesicovaginal...",C,C
1271,A 62-year-old patient has been hospitalized fo...,"{'A': 'Pseudomona aeruginosa', 'B': 'Streptoco...",E,A


In [72]:
len(cot[cot['answer_idx'] == cot['llm_answer']])/len(cot)

0.7195600942655145

In [73]:
len(cot[cot['llm_answer'].str.len() > 1])

30

In [74]:
rewrite = pd.read_csv('results_with_rephrasal_all.csv')
rewrite

,questions,options,answer_idx,llm_answer
0,A 32-year-old man with HIV comes to the physic...,"{'A': 'Secretion of interferon-α', 'B': 'Inter...",B,B
1,A one-day-old male is evaluated in the hospita...,"{'A': 'Duodenal atresia', 'B': 'Intestinal mal...",A,A
2,A 38-year-old woman presents to the office for...,"{'A': 'Body dysmorphic disorder', 'B': 'Schizo...",C,C
3,A 47-year-old woman is brought to the emergenc...,"{'A': 'COPD exacerbation', 'B': 'Diabetic keto...",E,E
4,A 10-year-old boy is brought to the emergency ...,{'A': 'Hypoxanthine-guanine phosphoribosyl tra...,D,C \nQuestion: Which specific biochemical path...
...,...,...,...,...
1268,A 16-month-old boy is brought to the physician...,{'A': 'Deficiency of cofactor for prolyl and l...,C,C
1269,A 21-year-old man comes to the physician becau...,"{'A': 'Wood lamp examination', 'B': 'Skin cult...",C,C
1270,"A 27-year-old woman, gravida 1, para 0, at 38 ...","{'A': 'Urge incontinence', 'B': 'Vesicovaginal...",C,C
1271,A 62-year-old patient has been hospitalized fo...,"{'A': 'Pseudomona aeruginosa', 'B': 'Streptoco...",E,B


In [75]:
len(rewrite[rewrite['llm_answer'].str.len() > 1])

47

In [76]:
len(rewrite[rewrite['answer_idx'] == rewrite['llm_answer']])/len(rewrite)

0.6716417910447762

In [77]:
rag = pd.read_csv('results_with_rag_all.csv')
rag

,questions,options,answer_idx,llm_answer,cot_output,faithfulness,rag_information
0,A 32-year-old man with HIV comes to the physic...,"{'A': 'Secretion of interferon-α', 'B': 'Inter...",B,B,"To answer the question, we need to identify th...",0.222222,**Relevant Segments**:\n\n1. **Document on Tub...
1,A one-day-old male is evaluated in the hospita...,"{'A': 'Duodenal atresia', 'B': 'Intestinal mal...",A,A,To determine the most likely etiology of the o...,0.133333,Documents relevant to the question about the o...
2,A 38-year-old woman presents to the office for...,"{'A': 'Body dysmorphic disorder', 'B': 'Schizo...",C,C,To determine the most consistent diagnosis ali...,0.333333,Here are the relevant segments extracted from ...
3,A 47-year-old woman is brought to the emergenc...,"{'A': 'COPD exacerbation', 'B': 'Diabetic keto...",E,E,To determine which option could explain the pa...,0.533333,1. **Hypoventilation**: This results in elevat...
4,A 10-year-old boy is brought to the emergency ...,{'A': 'Hypoxanthine-guanine phosphoribosyl tra...,D,D,To determine the correct option regarding the ...,0.400000,Here are the useful segments extracted in resp...
...,...,...,...,...,...,...,...
1268,A 16-month-old boy is brought to the physician...,{'A': 'Deficiency of cofactor for prolyl and l...,C,C,To determine the most likely underlying cause ...,0.916667,The relevant segments addressing the case of t...
1269,A 21-year-old man comes to the physician becau...,"{'A': 'Wood lamp examination', 'B': 'Skin cult...",C,D,To determine the correct option for confirming...,0.111111,The relevant segments that are useful to the q...
1270,"A 27-year-old woman, gravida 1, para 0, at 38 ...","{'A': 'Urge incontinence', 'B': 'Vesicovaginal...",C,C,To determine the correct answer to the questio...,0.100000,**Relevant Extracts:**\n\n1. **Document 111**:...
1271,A 62-year-old patient has been hospitalized fo...,"{'A': 'Pseudomona aeruginosa', 'B': 'Streptoco...",E,B,To determine the most likely causal microorgan...,0.000000,Here are the segments from the document that a...


In [78]:
len(rag[rag['answer_idx'] == rag['llm_answer']])/len(rag)

0.6865671641791045

In [87]:
rag['faithfulness'].mean()

0.356583141131353

In [89]:
rag[rag['answer_idx'] == rag['llm_answer']]['faithfulness'].mean()

0.3607419615230313

In [90]:
rag[rag['answer_idx'] != rag['llm_answer']]['faithfulness'].mean()

0.3474733440829148

In [82]:
len(rag[rag['llm_answer'].str.len() > 1])

35

In [83]:
whole_workflow = pd.read_csv('results_reph_rag_all.csv')
whole_workflow

,questions,options,answer_idx,llm_answer,cot_output,faithfulness,rag_information
0,A 32-year-old man with HIV comes to the physic...,"{'A': 'Secretion of interferon-α', 'B': 'Inter...",B,B,"To answer the question, we need to analyze the...",0.250000,The relevant segment of the document is as fol...
1,A one-day-old male is evaluated in the hospita...,"{'A': 'Duodenal atresia', 'B': 'Intestinal mal...",A,A** (Duodenal atresia).,To determine the most plausible underlying eti...,0.272727,The useful segments related to the question re...
2,A 38-year-old woman presents to the office for...,"{'A': 'Body dysmorphic disorder', 'B': 'Schizo...",C,C,"To find the correct answer, let's analyze the ...",1.000000,"**Relevant segments:**\n\n- ""The first impress..."
3,A 47-year-old woman is brought to the emergenc...,"{'A': 'COPD exacerbation', 'B': 'Diabetic keto...",E,E,To determine the correct option based on the c...,0.466667,**Key segments related to the patient's presen...
4,A 10-year-old boy is brought to the emergency ...,{'A': 'Hypoxanthine-guanine phosphoribosyl tra...,D,D,"To determine the correct option, we’ll analyze...",0.727273,The useful segments from the provided extract ...
...,...,...,...,...,...,...,...
1268,A 16-month-old boy is brought to the physician...,{'A': 'Deficiency of cofactor for prolyl and l...,C,C,To determine the underlying etiology contribut...,0.923077,The segments that are useful to the question i...
1269,A 21-year-old man comes to the physician becau...,"{'A': 'Wood lamp examination', 'B': 'Skin cult...",C,D,To determine the most appropriate diagnostic t...,0.090909,**Relevant Extract:** \n\n- **Pityriasis rosea...
1270,"A 27-year-old woman, gravida 1, para 0, at 38 ...","{'A': 'Urge incontinence', 'B': 'Vesicovaginal...",C,C,To determine the correct answer regarding the ...,0.333333,The relevant segments from the documents regar...
1271,A 62-year-old patient has been hospitalized fo...,"{'A': 'Pseudomona aeruginosa', 'B': 'Streptoco...",E,B,To determine the most probable etiological pat...,0.000000,The relevant extracted portion discussing pneu...


In [84]:
len(whole_workflow[whole_workflow['answer_idx'] == whole_workflow['llm_answer']])/len(whole_workflow)

0.6834249803613511

In [91]:
whole_workflow['faithfulness'].mean()

0.3367134265204726

In [92]:
whole_workflow[whole_workflow['answer_idx'] == whole_workflow['llm_answer']]['faithfulness'].mean()

0.34823209017365586

In [93]:
whole_workflow[whole_workflow['answer_idx'] != whole_workflow['llm_answer']]['faithfulness'].mean()

0.3118468325297295

In [53]:
len(whole_workflow[whole_workflow['llm_answer'].str.len() > 1])

35